In [ ]:
!pip install ultralytics==8.0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.5/314.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.2 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
import os

In [ ]:
model_path = '/content/yolo/best.pt'

In [ ]:
folder_path = '/content/raw'

In [ ]:
predictedList = []
def prediction_image():
  for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
      img = cv2.imread(os.path.join(folder_path, filename))
      images = cv2.imread(os.path.join(folder_path, filename))
      imagess = cv2.imread(os.path.join(folder_path, filename))
      model = YOLO(model_path)
      results = model.predict(source=img,conf=.70)
      if len(results[0].boxes) == 0: # Check if no bounding boxes are found
        continue # Skip to the next image

      boundingbox(results,images)
      #cv2_imshow(img)
      for result in results:
        predictedList.append(result)
        removeBackground(imagess, filename)
  cv2_imshow(img) # Display the annotated image

In [ ]:
def boundingbox(results,images):
  for result in results:
    for box in result.boxes.xyxy:
      x1, y1, x2, y2 = box.cpu().numpy().astype(int)
      cv2.rectangle(images, (x1, y1), (x2, y2), (0, 255, 0), 2) # Draw rectangle on the image
  #cv2_imshow(images)

In [ ]:
def masking(img, H,W,_):
  for result in predictedList:
    for j, mask in enumerate (result.masks.data):

      mask = mask.cpu().numpy() * 255  # Move tensor to CPU before converting
      mask = cv2.resize(mask, (W, H))
      #cv2.imwrite(f'mask_{j}.jpg', mask)
      last_mask = mask

  #cv2_imshow(mask)


In [ ]:
def removeBackground(img, filename):
  for result in predictedList:
      for j, mask in enumerate(result.masks.data):
          mask = mask.cpu().numpy()
          mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

          background = np.zeros_like(img)
          extracted_object = np.where(mask[..., None], img, background)

          output_dir = "Background_Removed"
          os.makedirs(output_dir, exist_ok=True)
          # Include the original filename in the output filename
          output_path = os.path.join(output_dir, f"{filename[:-4]}_object_{j}.jpg")
          cv2.imwrite(output_path, extracted_object)

In [ ]:
      prediction_image()

Output hidden; open in https://colab.research.google.com to view.

In [15]:
from google.colab import files

In [16]:
import shutil
shutil.make_archive('Background_Removed', 'zip', 'Background_Removed')
files.download('Background_Removed.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>